In [6]:
from decaylanguage.particle import Particle, Charge
from decaylanguage.particle.particle import getname

In [2]:
Particle.from_pdgid(211)

Particle(val=211, name='pi', mass=0.13957018)

```
* 1) ignore documentation lines that begin with an asterisk
* 2) in a FORTRAN program, process data lines with
*    FORMAT (BN, 4I8, 2(1X,E18.0, 1X,E8.0, 1X,E8.0), 1X,A21)
* 3)    column
*       1 -  8 \ Monte Carlo particle numbers as described in the "Review of
*       9 - 16 | Particle Physics". Charge states appear, as appropriate,
*      17 - 24 | from left-to-right in the order -, 0, +, ++.
*      25 - 32 / 
*           33   blank
*      34 - 51   central value of the mass (double precision)
*           52   blank
*      53 - 60   positive error
*           61   blank
*      62 - 69   negative error
*           70   blank
*      71 - 88   central value of the width (double precision)
*           89   blank
*      90 - 97   positive error
*           98   blank
*      99 -106   negative error
*          107   blank
*     108 -128   particle name left-justified in the field and
*                charge states right-justified in the field.
*                This field is for ease of visual examination of the file and
*                should not be taken as a standardized presentation of
*                particle names.
```

In [83]:
from decaylanguage.data import open_text
import pandas as pd
textfile = open_text('decaylanguage.data', 'mass_width_2018.mcd')
mcd_table = pd.read_fwf(textfile,
                            widths=(8,8,8,8,19,9,9,19,9,9,22),
                            names='I0 I1 I2 I3 Mass MassUpper MassLower Width WidthUpper WidthLower Name'.split(),
                            comment = "*")
names = mcd_table['Name'].str.split(expand=True)
mcd_table['Name'] =  names[0]
charges = names[1].str.split(',', expand=True)
filter_table = (charges[0] == '-1/3') | (charges[0] == '+2/3')
mcd_table = mcd_table[~filter_table]
charges = charges[~filter_table]

In [88]:
def filter_out_charge(table, column, charge):
    newtable = table.copy()
    newtable['Charge'] = charge
    newtable = newtable[~pd.isna(newtable[column])].copy()
    newtable.set_index(newtable[column].astype(int).rename("ID"), inplace=True)
    del newtable['I0'], newtable['I1'], newtable['I2'], newtable['I3']
    return newtable

In [101]:
pdg = Particle.pdg_table()

In [130]:
I3 = filter_out_charge(mcd_table, 'I3', charges[3])
I2 = filter_out_charge(mcd_table, 'I2', charges[2])
I1 = filter_out_charge(mcd_table, 'I1', charges[1])
I0 = filter_out_charge(mcd_table, 'I0', charges[0])
table = pd.concat((I0, I1, I2, I3)).sort_index()

In [131]:
index = table[table.Charge.isnull()].index
table.Charge = table.Charge.str.replace('-', '-1').replace('++', '2').replace('+', '1')
table.loc[index,'Charge'] = pdg.loc[index, 'Charge']
table.Charge = table.Charge.astype(int)

In [156]:
new_particles = set(table.index) - set(pdg.index)
table.loc[sorted(new_particles)]

,Mass,MassUpper,MassLower,Width,WidthUpper,WidthLower,Name,Charge
ID,,,,,,,,
21,0.00000,0.00000,-0.00000,0.000000e+00,0.000000e+00,-0.000000e+00,g,0
25,125.18000,0.16000,-0.16000,NaN,NaN,NaN,H,0
5332,6.04610,0.00170,-0.00170,5.900000e-25,1.000000e-25,-8.000000e-26,Omega(b),-1
10221,1.35000,0.15000,-0.15000,3.500000e-01,1.500000e-01,-1.500000e-01,f(0)(1370),0
10553,9.89930,0.00080,-0.00080,NaN,NaN,NaN,h(b)(1P),0
11112,1.86000,0.06000,-0.02000,2.500000e-01,7.000000e-02,-7.000000e-02,Delta(1900),-1
11212,1.86000,0.06000,-0.02000,2.500000e-01,7.000000e-02,-7.000000e-02,Delta(1900),0
12122,1.86000,0.06000,-0.02000,2.500000e-01,7.000000e-02,-7.000000e-02,Delta(1900),1
12222,1.86000,0.06000,-0.02000,2.500000e-01,7.000000e-02,-7.000000e-02,Delta(1900),2


In [157]:
old_particles = set(pdg.index) - set(table.index)
old_particles = [p for p in sorted(old_particles) if p > 0]
pdg.loc[old_particles]

,Mass,MassUpper,MassLower,Width,WidthUpper,WidthLower,I,G,J,P,C,A,Charge,Rank,Status,Name,Quarks,Latex
ID,,,,,,,,,,,,,,,,,,
551,9300.0,28.0,28.0,-1.000000e+00,-1.0,-1.0,0,1,0,-1,1,0,0,0,2,eta(b)(1S),bB,\eta_\mathrm{b}(1S)
4412,3518.9,0.9,0.9,2.007900e-08,-1.0,-1.0,?,0,?,5,0,1,1,1,2,Xi(cc),dcc,\Xi_{cc}^{+}
4422,3518.9,0.9,0.9,2.007900e-08,-1.0,-1.0,?,0,?,5,0,1,2,1,2,Xi(cc),ucc,\Xi_{cc}^{++}
5212,5807.8,2.7,2.7,-1.000000e+00,-1.0,-1.0,1,0,1/2,1,0,1,0,3,0,Sigma(b),udb,\Sigma_\mathrm{b}^{0}
5214,5829.0,3.4,3.4,-1.000000e+00,-1.0,-1.0,1,0,3/2,1,0,1,0,3,0,Sigma(b),,\Sigma_\mathrm{b}^{*0}
9981,9990.0,10.0,10.0,9.900000e+09,30.0,30.0,0,1,0,1,1,0,0,0,0,NonResS,??,0^{+}
9983,9990.0,10.0,10.0,9.900000e+09,30.0,30.0,1/2,0,1,1,0,1,0,0,0,NonResA,??,1^{+}
9985,9990.0,10.0,10.0,9.900000e+09,30.0,30.0,0,1,2,1,1,0,0,0,2,NonResT,??,2^{+}
9986,9990.0,10.0,10.0,9.900000e+09,30.0,30.0,0,1,2,-1,1,0,0,0,2,NonResPT,??,2^{-}
